In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
# imports
import numpy as np
import random as rd
import math
import shutil
import datetime
from scipy.special import gamma
from scipy.optimize import minimize, Bounds

# setting up numpy random
np.random.seed()
generator = np.random.default_rng()

In [4]:
class MLSL_new():
    """ Multi-level single linkage algorithm.

    Parameters:
    --------------
    budget : int
             The budget for function evaluations.

    Attributes:
    --------------
    d : int
        Number of problem variables, dimensionality.

    x_opt : array
            The best found solution.

    f_opt : float
            Function value for the best found solution.

    pop : array
          Matrix holding all current solution candidates or points.

    n_points : int
               The number of new solutions per iteration.

    gamma: float
           Factor determining the size of the reduced sample.

    k : int
        The current iteration number.

    zeta : float
           The scaling factor for updating the critical distance.

    xr : array
         Matrix holding the reduced sample points.

    fr : array
         Array holding the fitness values for points in the reduced sample.

    rk : float
         The critical distance rk.

    lebesgue : float
               The Lebesgue measure of distance.

    """

    def __init__(self, budget):
        self.budget = budget
        self.d = 1
        self.x_opt = None
        self.f_opt = np.inf
        self.pop = None
        self.n_points = 1
        self.gamma = 0.1
        self.k = 1
        self.zeta = 2.0
        self.xr = None
        self.fr = None
        self.rk = 0
        self.lebesgue = 0

    def calc_rk(self):
        """ Calculates the critical distance depending on current iteration and population.

        Parameters:
        -------------
        None

        Returns:
        -------------
        rk : float
             The critical distance rk

        """
        kN = self.k * len(self.pop)
        rk = (1 / math.sqrt(np.pi)) * math.pow((gamma(1 + (self.d / 2))
                                    * self.lebesgue * (self.zeta * math.log1p(kN)) / kN), (1 / self.d))

        return rk

    def __call__(self, func):
        """ Run the MLSL algorithm.

        Parameters:
        ------------
        func : object
               Function to be optimized.

        Returns:
        ------------
        x_opt : array
                The best found solution.

        f_opt: float
               The fitness value for the best found solution.

        """

        # Set parameters depending on function characteristics
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        local_budget = 0.1 * eval_budget     # factor 0.1 according to original BBOB submission
        bounds = Bounds(func.lowerbound, func.upperbound)
        self.n_points = 50 * self.d    # 50 points according to original BBOB submission
        self.lebesgue = math.sqrt(100 * self.d)

        # Initialize reduced sample and (re)set iteration counter to 1
        self.pop = []
        f = []
        x_star = []
        f_star = []
        self.k = 1

        # Start iteration
        while func.evaluations < eval_budget and not func.best_so_far_precision <= 1e-8:

            # Sample new points
            for i in range(0, self.n_points):
                new_point = np.zeros(self.d)
                for j in range(0, self.d):
                    new_point[j] = generator.uniform(low=-5, high=5)
                self.pop.append(new_point)
                f.append(func(new_point))

            # Extract reduced sample xr
            self.xr = np.zeros((math.ceil(self.gamma * self.k * self.n_points), self.d))
            m = np.hstack((np.asarray(self.pop), np.expand_dims(np.asarray(f), axis=1)))
            sorted_m = m[np.argsort(m[:, self.d])]
            self.xr = sorted_m[0:len(self.xr), 0:self.d]
            self.fr = sorted_m[0:len(self.xr), self.d]

            # Update rk
            self.rk = self.calc_rk()

            # Check critical distance and fitness differences in xr
            for i in range(0, len(self.xr)):
                cond = False
                for j in range(0, len(self.xr)):
                    if j == i:
                        continue
                    if self.fr[j] < self.fr[i]:
                        cond = np.linalg.norm(self.xr[j] - self.xr[i]) < self.rk
                    if cond:
                        break

                # If there is no point with better fitness in critical distance, start local search
                if not cond:
                    solution = minimize(func, self.xr[i], method='Powell', bounds=bounds,
                                        options={'ftol': 1e-8, 'maxfev': local_budget})
                    x_star.append(solution.x)
                    f_star.append(solution.fun)
                    local_budget = local_budget - solution.nfev
                    if local_budget < 0:
                        local_budget = 0

            self.k = self.k+1

        # Get best point from x_star
        n = np.hstack((np.asarray(x_star), np.expand_dims(np.asarray(f_star), axis=1)))
        sorted_n = n[np.argsort(n[:, self.d])]
        self.x_opt = sorted_n[0, 0:self.d]
        self.f_opt = sorted_n[0, self.d]

        return self.x_opt, self.f_opt


65:80: E501 line too long (83 > 79 characters)
79:37: E128 continuation line under-indented for visual indent
79:80: E501 line too long (103 > 79 characters)
104:80: E501 line too long (95 > 79 characters)
106:80: E501 line too long (88 > 79 characters)
128:80: E501 line too long (88 > 79 characters)
129:80: E501 line too long (88 > 79 characters)
144:80: E501 line too long (80 > 79 characters)
148:80: E501 line too long (99 > 79 characters)
150:80: E501 line too long (89 > 79 characters)
151:80: E501 line too long (87 > 79 characters)
161:80: E501 line too long (87 > 79 characters)


In [6]:
fIDs = []
for fid in range(1, 25):
    fIDs.append(fid)
dIDs = [2, 3, 5, 10, 20, 40]

In [7]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1], dIDs, 15)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  "MLSL/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([MLSL_new(10000)])
#create a ZIP file for IOHprofiler upload
shutil.make_archive(path + experiment_name,"zip", path + experiment_name)

Running single-threaded
1 282 5.928782740801667e-30
1 204 7.346267179870672e-30
1 256 7.346267179870672e-30
1 256 5.928782740801667e-30
1 308 5.928782740801667e-30
1 308 7.346267179870672e-30
1 334 5.928782740801667e-30
1 282 7.346267179870672e-30
1 256 4.979684464207637e-30
1 308 5.928782740801667e-30
1 204 5.928782740801667e-30
1 282 4.979684464207637e-30
1 256 4.979684464207637e-30
1 204 5.928782740801667e-30
1 204 5.928782740801667e-30
1 644 4.4373425918681914e-30
1 682 8.578862344278503e-30
1 720 7.161377905209498e-30
1 720 2.428212473883427e-30
1 720 2.070759876205156e-30
1 682 6.212279628615468e-30
1 644 1.4791141972893971e-30
1 720 2.428212473883427e-30
1 682 6.212279628615468e-30
1 720 3.8456969129524326e-30
1 720 3.8456969129524326e-30
1 644 4.4373425918681914e-30
1 720 1.4791141972893971e-30
1 682 4.4373425918681914e-30
1 682 6.212279628615468e-30
1 1800 5.038849032099213e-29
1 1800 4.343665359373196e-29
1 1800 3.73353075299132e-29
1 1800 4.50636792107503e-29
1 1800 3.853092

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/MLSL/experiment5-10-163915.zip'

3:25: E231 missing whitespace after ','
4:80: E501 line too long (101 > 79 characters)
6:80: E501 line too long (135 > 79 characters)
7:18: E222 multiple spaces after operator
10:1: E265 block comment should start with '# '
11:43: E231 missing whitespace after ','
